In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("imdb_animated_movies_clean.csv")
df.head()

,name,rating,year,genre_a,genre_b,runtime_minutes,story_desc,avg_rating_score,votescore,metacritic,votes,gross_dollar,profit_dollar,avg_dollar_budget_per_minute,budget_est_dollar,orig_bgt_currency,director
0,9,PG-13,2009,Action,Adventure,79,A rag doll that awakens in a postapocalyptic f...,65.0,70.0,60.0,140989,31740000,1740000,379746.84,30000000,dollar,Shane Acker
1,A Bug's Life,G,1998,Adventure,Comedy,95,"A misfit ant, looking for ""warriors"" to save h...",74.5,72.0,77.0,293639,162800000,42800000,1263157.89,120000000,dollar,John Lasseter
2,A Christmas Carol,PG,2009,Adventure,Comedy,96,An animated retelling of Charles Dickens' clas...,61.5,68.0,55.0,115942,137860000,-62140000,2083333.33,200000000,dollar,Robert Zemeckis
3,A Goofy Movie,G,1995,Adventure,Comedy,78,When Max makes a preposterous promise to a gir...,61.0,69.0,53.0,55556,35350000,17350000,230769.23,18000000,dollar,Kevin Lima
4,A Scanner Darkly,R,2006,Comedy,Crime,100,An undercover cop in a not-too-distant future ...,71.5,70.0,73.0,112638,5500000,-3200000,87000.00,8700000,dollar,Richard Linklater


In [4]:
df.rating.value_counts()

PG           189
G             86
PG-13         18
R             13
Not Rated      6
TV-MA          4
TV-PG          3
TV-14          2
Name: rating, dtype: int64

In [2]:
df.genre_a.value_counts()

Adventure    212
Action        62
Drama         22
Comedy        15
Family         5
Biography      3
Sci-Fi         1
Crime          1
Name: genre_a, dtype: int64

In [3]:
df.genre_b.value_counts()

Comedy       175
Adventure     54
Family        32
Drama         31
Fantasy       16
Crime          5
Romance        2
Horror         1
War            1
Mystery        1
Sci-Fi         1
Name: genre_b, dtype: int64